In [ ]:
!pip install pythainlp

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string
from pythainlp.tokenize import word_tokenize
from sklearn.naive_bayes import MultinomialNB


In [ ]:
df_fake = pd.read_csv("./Fake.csv")
df_true = pd.read_csv("./True.csv")

In [ ]:
df_fake.head()

,title,date
0,"ออมสินปล่อยสินเชื่อกู้ง่ายจัดให้ วงเงิน 5,000-...",6/5/1905
1,ตู้กดเงิน ธกส. สาขาบ้านดงบังใต้ กดหน้าจอที่ขึ้...,13/5/1905
2,ขนส่งเปิดทำใบขับขี่ออนไลน์ สำหรับผู้ที่ไม่เคยม...,13/5/1905
3,เปิดรับสมัครคนพับถุงกระดาษและแพ็กของ ทางเพจ Or...,13/5/1905
4,มีการแอบเอาข้าวเก่า 10 ปี ในโกดังมาขายปนกับข้าวดี,13/5/1905


In [ ]:
df_true.head()

,title,date
0,GISTDA เตือนไทยควรเตรียมรับมือพายุสนามแม่เหล็ก...,6/5/1905
1,มีมติเห็นชอบมาตรการลดภาระค่าใช้จ่ายด้านพลังงาน...,13/5/1905
2,นอนหลับหลัง 4 ทุ่ม อันตรายต่อสุขภาพ และมีความเ...,13/5/1905
3,ลดค่าโอนและค่าจดจำนองอสังหาริมทรัพย์ ที่ราคาไม...,13/5/1905
4,อุทยานสิรินาถ ปิดการท่องเที่ยวบริเวณเกาะปลิง แ...,13/5/1905


In [ ]:
df_fake["class"] = 0
df_true["class"] = 1

In [ ]:
df_fake.shape, df_true.shape

((100, 3), (100, 3))

In [ ]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(99,89,-1):
    df_fake.drop([i], axis = 0, inplace = True)

df_true_manual_testing = df_true.tail(10)
for i in range(99,89,-1):
    df_true.drop([i], axis = 0, inplace = True)

In [ ]:
df_fake_manual_testing["class"] = 0
df_true_manual_testing["class"] = 1

<ipython-input-70-3aaf8ec2aad1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing["class"] = 0
<ipython-input-70-3aaf8ec2aad1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manual_testing["class"] = 1


In [ ]:
df_fake_manual_testing.head(10)

,title,date,class
90,ลุกจากที่นอนกะทันหัน ทำให้กระดูกกะโหลกศีรษะแตก...,13/5/1905,0
91,กรุงไทยเปิดเพจชื่อสินเชื่อเงินสด.,1/5/1905,0
92,เปิดพัดลมจ่อเด็ก เสี่ยงเป็นปอดอักเสบ,13/5/1905,0
93,ไม่เปิดหน้าต่างระบายความร้อนในรถ สารพิษของเบนซ...,13/5/1905,0
94,การบินไทยรับสมัครงาน ผ่านเพจฝ่ายสื่อสารองค์กรก...,10/5/1905,0
95,บัญชีไลน์ของผู้ว่าการธนาคารแห่งประเทศไทยชักชวน...,10/5/1905,0
96,ปัสสาวะกลางคืนแล้วไม่ดื่มน้ำทดแทน ทำให้เกิดภาว...,26/4/1905,0
97,เพจเฟซบุ๊กชื่อ “ศูนย์พัฒนาฝีมือแรงงาน” เป็นเพจ...,26/4/1905,0
98,ไม่ใส่รองเท้าในพื้นที่ปูกระเบื้องหรือหินอ่อน ...,10/5/1905,0
99,"สินเชื่อกรุงไทย กู้ง่ายผ่อนแสนละ 1,866.67 บาทต...",30/4/1905,0


In [ ]:
df_true_manual_testing.head(10)

,title,date,class
90,อนุมัติงบช่วยเหลือผู้สูงอายุทั่วประเทศ เงินสงเ...,13/5/1905,1
91,รัฐบาลเตรียมกระตุ้นการท่องเที่ยวทางบก Drive To...,1/5/1905,1
92,PM 2.5 ส่งผลกระทบต่อความดันและหัวใจ,13/5/1905,1
93,โรคไข้นกแก้วคล้ายไข้หวัดใหญ่-ปอดอักเสบรุนแรง,13/5/1905,1
94,เกาหลีใต้เตรียมเปิดรับสมัครแรงงานหญิงไทย อายุ ...,10/5/1905,1
95,ธอส. เพิ่มโปรโมชันสำหรับคนอยากมีบ้าน ผ่อนล้านล...,10/5/1905,1
96,คนที่ดื่มแอลกอฮอลล์หนักและสูบบุหรี่จัด ทำให้เล...,26/4/1905,1
97,นายกฯ ตั้งเป้าหมายขับเคลื่อนไทยเป็นศูนย์กลางกา...,26/4/1905,1
98,สภาวะเอลนีโญส่งผลให้อุณหภูมิโลก และน้ำทะเลสูงข...,10/5/1905,1
99,ค่าฝุ่น PM 2.5 ในภาคเหนือสูง เพราะเกิดจากหมอกค...,30/4/1905,1


In [ ]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

# Merging True and Fake Dataframes

In [ ]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

,title,date,class
0,"ออมสินปล่อยสินเชื่อกู้ง่ายจัดให้ วงเงิน 5,000-...",6/5/1905,0
1,ตู้กดเงิน ธกส. สาขาบ้านดงบังใต้ กดหน้าจอที่ขึ้...,13/5/1905,0
2,ขนส่งเปิดทำใบขับขี่ออนไลน์ สำหรับผู้ที่ไม่เคยม...,13/5/1905,0
3,เปิดรับสมัครคนพับถุงกระดาษและแพ็กของ ทางเพจ Or...,13/5/1905,0
4,มีการแอบเอาข้าวเก่า 10 ปี ในโกดังมาขายปนกับข้าวดี,13/5/1905,0
5,กรมอุตุฯ เตือนเฝ้าระวังพายุไซโคลนก่อตัว ในพื้น...,13/5/1905,0
6,ผู้สูงอายุ 60 ปีขึ้นไปที่เสียชีวิต ทายาทจะได้ร...,13/5/1905,0
7,ดื่มแอลกอฮอล์ติดต่อกัน ช่วยเสริมสร้างระบบภูมิค...,13/5/1905,0
8,ปวดประจำเดือน สาเหตุมาจากลมแทรกในมดลูก มดลูกบี...,13/5/1905,0
9,กรมอุตุฯ เตือนเฝ้าระวังพายุไซโคลนก่อตัว ในพื้น...,13/5/1905,0


In [ ]:
df_merge.columns

Index(['title', 'date', 'class'], dtype='object')

In [ ]:
df = df_merge.drop(["date"], axis = 1)

In [ ]:
df.isnull().sum()

title    0
class    0
dtype: int64

In [ ]:
df = df.sample(frac = 1)

In [ ]:
df.head()

,title,class
74,ทำใบขับขี่หรือต่อใบขับขี่ ผ่านเพจ DLT รับต่อแล...,0
22,สินเชื่อเงินกู้ฉุกเฉิน กู้ได้ทุกอาชีพ เพียงแค่...,0
21,"เงินกู้ฉุกเฉิน ปล่อยกู้ 5,000-1,000,000 บาท ผ่...",0
46,"ออมสินปล่อยสินเชื่อกู้ง่ายจัดให้ วงเงิน 5,000-...",0
31,เปิดลงทะเบียนเข้าร่วมมาตรการบรรเทาภาระค่าไฟฟ้า...,1


In [ ]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [ ]:
df.columns

Index(['title', 'class'], dtype='object')

In [ ]:
df.head()

,title,class
0,ทำใบขับขี่หรือต่อใบขับขี่ ผ่านเพจ DLT รับต่อแล...,0
1,สินเชื่อเงินกู้ฉุกเฉิน กู้ได้ทุกอาชีพ เพียงแค่...,0
2,"เงินกู้ฉุกเฉิน ปล่อยกู้ 5,000-1,000,000 บาท ผ่...",0
3,"ออมสินปล่อยสินเชื่อกู้ง่ายจัดให้ วงเงิน 5,000-...",0
4,เปิดลงทะเบียนเข้าร่วมมาตรการบรรเทาภาระค่าไฟฟ้า...,1


# Creating a function to process th texts

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
def wordopt_thai(text):
    words = word_tokenize(text,engine='newmm')
    result = ' '.join(words)


    return result

In [ ]:
df["title"] = df["title"].apply(wordopt_thai)

In [ ]:
df

,title,class
0,ทำ ใบขับขี่ หรือ ต่อ ใบขับขี่ ผ่าน เพจ DLT...,0
1,สินเชื่อ เงินกู้ ฉุกเฉิน กู้ ได้ ทุก อาชีพ ...,0
2,"เงินกู้ ฉุกเฉิน ปล่อย กู้ 5,000 - 1,000,00...",0
3,ออมสิน ปล่อย สินเชื่อ กู้ ง่าย จัด ให้ วงเงิ...,0
4,เปิด ลงทะเบียน เข้าร่วม มาตรการ บรรเทา ภาระ ค่...,1
...,...,...
175,เขื่อน ป่า สัก ชล สิทธิ์ ใน เดือน เม.ย. มี น...,1
176,นอนตะแคง ซ้าย ช่วย ลด อาการ แสบร้อน กลา งอก ...,1
177,ลด ค่า โอน และ ค่า จด จำนอง อสังหาริมทรัพย์ ...,1
178,ไลน์ ผู้เชี่ยวชาญ ด้าน การ วิเคราะห์ และ ประ...,0


# Defining dependent and independent variables

In [ ]:
x = df["title"]
y = df["class"]

# Splitting Training and Testing

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# Convert text to vectors

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer(max_features=5000)
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [ ]:
pred_lr=LR.predict(xv_test)


In [ ]:
LR.score(xv_test, y_test)

0.7333333333333333

In [ ]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.78      0.64      0.70        22
           1       0.70      0.83      0.76        23

    accuracy                           0.73        45
   macro avg       0.74      0.73      0.73        45
weighted avg       0.74      0.73      0.73        45



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid for LogisticRegression
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 200, 300]
}

# Initialize the model
model = LogisticRegression(random_state=0)

# Grid Search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(xv_train, y_train)

# Best parameters and model
print("Best Parameters:", grid_search.best_params_)
best_model_LR = grid_search.best_estimator_

# Evaluation
y_pred = best_model_LR.predict(xv_test)
print("Optimized Accuracy:", accuracy_score(y_test, y_pred))
print("Optimized Classification Report:\n", classification_report(y_test, y_pred))


Fitting 3 folds for each of 300 candidates, totalling 900 fits
Best Parameters: {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
Optimized Accuracy: 0.7333333333333333
Optimized Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.64      0.70        22
           1       0.70      0.83      0.76        23

    accuracy                           0.73        45
   macro avg       0.74      0.73      0.73        45
weighted avg       0.74      0.73      0.73        45



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
405 fits failed out of a total of 900.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Sol

# Decision Tree Classification

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [ ]:
pred_dt = DT.predict(xv_test)

In [ ]:
DT.score(xv_test, y_test)

0.6888888888888889

In [ ]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.72      0.59      0.65        22
           1       0.67      0.78      0.72        23

    accuracy                           0.69        45
   macro avg       0.69      0.69      0.69        45
weighted avg       0.69      0.69      0.69        45



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid with adjusted values
param_grid = {
    'criterion': ['gini', 'entropy'],  # Added criterion
    'splitter': ['best', 'random'],  # Added splitter
    'max_depth': [None, 10, 20, 30, 50, 100],  # Expanded max_depth
    'min_samples_split': [2, 5, 10, 15, 20],  # Expanded min_samples_split
    'min_samples_leaf': [1, 2, 4, 8, 10],  # Expanded min_samples_leaf
    'max_features': ['auto', 'sqrt', 'log2'],  # Adjusted max_features
    'random_state': [0],  # Fixed random_state
}

# Initialize the model
model = DecisionTreeClassifier()

# Grid Search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(xv_train, y_train)

# Best parameters and model
print("Best Parameters:", grid_search.best_params_)
best_model_DT = grid_search.best_estimator_

# Evaluation
y_pred = best_model_DT.predict(xv_test)
print("Optimized Accuracy:", accuracy_score(y_test, y_pred))
print("Optimized Classification Report:\n", classification_report(y_test, y_pred))


Fitting 3 folds for each of 1800 candidates, totalling 5400 fits
Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 0, 'splitter': 'best'}
Optimized Accuracy: 0.5555555555555556
Optimized Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.64      0.58        22
           1       0.58      0.48      0.52        23

    accuracy                           0.56        45
   macro avg       0.56      0.56      0.55        45
weighted avg       0.56      0.56      0.55        45



/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(


# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [ ]:
pred_gbc = GBC.predict(xv_test)

In [ ]:
GBC.score(xv_test, y_test)

0.7555555555555555

In [ ]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       0.82      0.64      0.72        22
           1       0.71      0.87      0.78        23

    accuracy                           0.76        45
   macro avg       0.77      0.75      0.75        45
weighted avg       0.77      0.76      0.75        45



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.05, 0.01],  # Adding learning rate
    'max_depth': [3, 5, 7],  # Narrowing down max_depth range
    'min_samples_split': [2, 5],  # Reducing min_samples_split options
    'min_samples_leaf': [1, 2],
    'subsample': [0.8, 1.0],  # Adding subsample
    'max_features': ['auto', 'sqrt', 'log2'],
    'random_state': [0]  # Keeping random_state constant
}

# Initialize the model
model = GBC()

# Grid Search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(xv_train, y_train)

# Best parameters and model
print("Best Parameters:", grid_search.best_params_)
best_model_GBC = grid_search.best_estimator_

# Evaluation
y_pred = best_model_GBC.predict(xv_test)
print("Optimized Accuracy:", accuracy_score(y_test, y_pred))
print("Optimized Classification Report:\n", classification_report(y_test, y_pred))


Fitting 3 folds for each of 648 candidates, totalling 1944 fits


KeyboardInterrupt: 

# MultinoMiaLNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

MNB = MultinomialNB()
MNB.fit(xv_train, y_train)

MultinomialNB()

In [ ]:
pred_mnb = MNB.predict(xv_test)

In [ ]:
MNB.score(xv_test, y_test)

0.6888888888888889

In [ ]:
print(classification_report(y_test, pred_mnb))

              precision    recall  f1-score   support

           0       0.70      0.64      0.67        22
           1       0.68      0.74      0.71        23

    accuracy                           0.69        45
   macro avg       0.69      0.69      0.69        45
weighted avg       0.69      0.69      0.69        45



In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid for MultinomialNB
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0],  # Adjust the alpha smoothing parameter
    'fit_prior': [True, False]  # Whether to learn class prior probabilities or not
}

# Initialize the Multinomial Naive Bayes model
model = MultinomialNB()

# Grid Search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(xv_train, y_train)

# Best parameters and model
print("Best Parameters:", grid_search.best_params_)
best_model_MNB = grid_search.best_estimator_

# Evaluation
y_pred = best_model_MNB.predict(xv_test)
print("Optimized Accuracy:", accuracy_score(y_test, y_pred))
print("Optimized Classification Report:\n", classification_report(y_test, y_pred))


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Parameters: {'alpha': 1.0, 'fit_prior': False}
Optimized Accuracy: 0.7111111111111111
Optimized Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.64      0.68        22
           1       0.69      0.78      0.73        23

    accuracy                           0.71        45
   macro avg       0.71      0.71      0.71        45
weighted avg       0.71      0.71      0.71        45



# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [ ]:
pred_rfc = RFC.predict(xv_test)

In [ ]:
RFC.score(xv_test, y_test)

0.7777777777777778

In [ ]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.80      0.73      0.76        22
           1       0.76      0.83      0.79        23

    accuracy                           0.78        45
   macro avg       0.78      0.78      0.78        45
weighted avg       0.78      0.78      0.78        45



In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize the model
model = RandomForestClassifier(random_state=0)

# Grid Search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(xv_train, y_train)

# Best parameters and model
print("Best Parameters:", grid_search.best_params_)
best_model_RFC = grid_search.best_estimator_

# # Evaluation
y_pred = best_model_RFC.predict(xv_test)
print("Optimized Accuracy:", accuracy_score(y_test, y_pred))
print("Optimized Classification Report:\n", classification_report(y_test, y_pred))


Fitting 3 folds for each of 648 candidates, totalling 1944 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best Parameters: {'bootstrap': True, 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Optimized Accuracy: 0.7555555555555555
Optimized Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.68      0.73        22
           1       0.73      0.83      0.78        23

    accuracy                           0.76        45
   macro avg       0.76      0.75      0.75        45
weighted avg       0.76      0.76      0.75        45



# Model Testing

In [ ]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = best_model_LR.predict(new_xv_test)
    pred_DT = best_model_DT.predict(new_xv_test)
    pred_GBC = best_model_GBC.predict(new_xv_test)
    pred_RFC = best_model_RFC.predict(new_xv_test)
    pred_MNB = best_model_MNB.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {} \nMNB Predictgion: {}".format(output_lable(pred_LR[0]),output_lable(pred_DT[0]),
                                                                                                              output_lable(pred_GBC[0]),
                                                                                                              output_lable(pred_RFC[0]),
                                                                                                              output_lable(pred_MNB[0])))

In [ ]:
news = str(input())
manual_testing(news)

กทม. เตรียมติดตั้งตู้น้ำดื่มฟรีทั่วกรุงเทพฯ 200 จุด


LR Prediction: Not A Fake News 
DT Prediction: Not A Fake News 
GBC Prediction: Not A Fake News 
RFC Prediction: Not A Fake News 
MNB Predictgion: Not A Fake News


# Export Model and vectorizer

In [ ]:
import joblib

joblib.dump(RFC, 'fake_news_model.pkl')
joblib.dump(vectorization, 'vectorizer.pkl')

['vectorizer.pkl']

In [ ]:
!python --version

Python 3.10.12


In [ ]:
import sklearn
print(sklearn.__version__)

1.2.2


In [ ]:
model = joblib.load('./fake_news_model.pkl')
vectorizer = joblib.load('./vectorizer.pkl')
def preprocess_text(text):
    tokens = word_tokenize(text, engine='newmm')
    return ' '.join(tokens)
data = str(input())
preprocessed_text = preprocess_text(data)
transformed_text = vectorizer.transform([preprocessed_text])
prediction = model.predict(transformed_text)
if prediction[0] == 1

print(prediction)

กทม. เตรียมติดตั้งตู้น้ำดื่มฟรีทั่วกรุงเทพฯ 200 จุด
[1]


In [ ]:
prediction[0]

1